In [7]:
from src.resolve_init import GetResolve
from pprint import pprint
from dftt_timecode import DfttTimecode

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()

end_frame = DfttTimecode(current_timeline.GetEndFrame(), "auto", 24, False, True)
end_frame.timecode_output("smpte")

'02:08:59:02'